In [60]:
import pandas as pd
import numpy as np
from time import sleep
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


In [61]:
#Librerias de Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [62]:
ciudades = ['santiago de compostela', 'barcelona']

In [63]:
def abrir_web(ciudad, pagina, fecha):

    driver = webdriver.Chrome()

    url = f"https://www.civitatis.com/es/{ciudad}/?pagina={pagina}&fromDate={fecha}&toDate={fecha}"
    driver.get(url)
    driver.maximize_window()

    try:
        # Esperamos a que aparezca el filtro para que la pagina carge bien
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'a-filter--applied'))
        )
    except Exception as e:
        print(f"Ha habido un error esperando la página: {e}")
    
    sleep(1)

    contenido_html = driver.page_source
    sopa = BeautifulSoup(contenido_html, 'html.parser')
    
    driver.close()

    return sopa, url

In [64]:
sopa = abrir_web('barcelona', '3', f'2024-11-24')[0]
categorias = sopa.findAll('div', {'class': 'o-search-list__item'})


In [65]:

nombres = []
puntuaciones = []
precios = []
descripciones = []
links = []

for pagina in tqdm(range(1, 7)):
    sopa = abrir_web('barcelona', pagina, f'2024-11-24')[0]
    categorias = sopa.findAll('div', {'class': 'o-search-list__item'})
    for categoria in categorias:
        
        try:
            nombres.append(categoria.find('h2', {"class": "comfort-card__title"}).text.strip())
        except:
            nombres.append(np.nan)
        try:
            puntuaciones.append(categoria.find('span', {"class": "m-rating--text"}).text.replace('/ 10', '').replace(',', '.').strip())    
        except:
            puntuaciones.append(np.nan)
        try:
            precios.append(categoria.find('span', {"class": "comfort-card__price__text"}).text.replace(' €', '').strip())
        except:
            precios.append(np.nan)    
        try:
            descripciones.append(categoria.find('div', {"class": "comfort-card__text l-list-card__text"}).text.replace('\xa0',' ').strip())
        except:
            descripciones.append(np.nan)
        try:
            links.append(f"https://www.civitatis.com/{categoria.find('a', {'class': 'ga-trackEvent-element _activity-link'}).get('href')}")
        except:
            links.append(np.nan)
    sleep(2)
dicionario_categorías = {'Nombre': nombres, 'Puntuación': puntuaciones, 'Precio (€)': precios, 'Descripción': descripciones, 'Link': links}
    
        
    
    
        
    
    
    
    
    


100%|██████████| 6/6 [01:03<00:00, 10.59s/it]


In [66]:
nombres

['Visita de la Sagrada Familia sin colas',
 'Free tour por Barcelona',
 'Excursión a Montserrat con tren cremallera',
 'Visita guiada por el Parque Güell',
 'Sagrada Familia con subida a las torres',
 'Free tour de Gaudí y la Barcelona modernista',
 'Traslados en Barcelona',
 'Entrada a la Casa Batlló',
 'Autobús turístico de Barcelona',
 'Free tour nocturno por el Barrio Gótico',
 'Tour de los misterios y leyendas de Barcelona',
 'Entrada al Spotify Camp Nou',
 'Excursión a Montserrat + Visita a una bodega',
 'Free tour de los misterios y leyendas del Barrio Gótico',
 'Tour privado por Barcelona ¡Tú eliges!',
 'Entrada a La Pedrera',
 'Tour en tuk tuk por Barcelona',
 'Paseo en catamarán al atardecer con música en directo',
 'Teleférico de Montjuïc',
 'Excursión a Tarragona y Sitges',
 'Visita de la Sagrada Familia sin colas',
 'Free tour por Barcelona',
 'Excursión a Montserrat con tren cremallera',
 'Visita guiada por el Parque Güell',
 'Sagrada Familia con subida a las torres',
 'F

In [67]:
puntuaciones

['8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8',
 '8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8',
 '8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8',
 '8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8',
 '8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8',
 '8.6',
 '9.5',
 '8.4',
 '7.7',
 '7.8',
 '9.7',
 nan,
 '9.3',
 '8.7',
 '9.6',
 '9.1',
 '8.1',
 '10',
 '9.4',
 '9.4',
 '9.1',
 '9.5',
 '9',
 '8.6',
 '8.8']

In [68]:
precios

['55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '16',
 '79,90',
 '¡Gratis!',
 '160',
 '29',
 '53,40',
 '25,50',
 '16',
 '98,50',
 '55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '16',
 '79,90',
 '¡Gratis!',
 '160',
 '29',
 '53,40',
 '25,50',
 '16',
 '98,50',
 '55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '16',
 '79,90',
 '¡Gratis!',
 '160',
 '29',
 '53,40',
 '25,50',
 '16',
 '98,50',
 '55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '16',
 '79,90',
 '¡Gratis!',
 '160',
 '29',
 '53,40',
 '25,50',
 '16',
 '98,50',
 '55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '16',
 '79,90',
 '¡Gratis!',
 '160',
 '29',
 '53,40',
 '25,50',
 '16',
 '98,50',
 '55',
 '¡Gratis!',
 '57',
 '25,60',
 '70',
 '¡Gratis!',
 '46',
 '35',
 '33',
 '¡Gratis!',
 '14',
 '

In [69]:
descripciones

['Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.',
 'El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.',
 'En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.',
 'En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.',
 'En esta visita a la Sagrada Familia conoceremos el lugar más famoso de Barcelona y subiremos a una de sus torres. ¡Tendremos una vista inmejorable de la ciudad!',
 'Acompañadnos a visitar el corazón de Barcelona en este free tour de Gaudí que nos llevará a descubrir los principales símbolos del modernismo en España.',
 'Servicio privado, puerta a puerta y al mejor precio. Un chó

In [70]:
links

['https://www.civitatis.com//es/barcelona/visita-guiada-sagrada-familia/',
 'https://www.civitatis.com//es/barcelona/free-tour-barcelona/',
 'https://www.civitatis.com//es/barcelona/excursion-montserrat/',
 'https://www.civitatis.com//es/barcelona/visita-guiada-parque-guell/',
 'https://www.civitatis.com//es/barcelona/visita-sagrada-familia-torres/',
 'https://www.civitatis.com//es/barcelona/tour-barcelona-modernista/',
 nan,
 'https://www.civitatis.com//es/barcelona/entrada-casa-batllo/',
 'https://www.civitatis.com//es/barcelona/autobus-turistico-barcelona/',
 'https://www.civitatis.com//es/barcelona/tour-nocturno-barrio-gotico/',
 'https://www.civitatis.com//es/barcelona/tour-misterios-leyendas-barcelona/',
 'https://www.civitatis.com//es/barcelona/camp-nou-experience/',
 'https://www.civitatis.com//es/barcelona/tour-tapas-vinos-montserrat/',
 'https://www.civitatis.com//es/barcelona/free-tour-misterios-leyendas-barrio-gotico/',
 'https://www.civitatis.com//es/barcelona/tour-privado

In [71]:
df_civitatis = pd.DataFrame({
    'Nombre': nombres,
    'Puntuación': puntuaciones,
    'Precio (€)': precios,
    'Descripción': descripciones,
    'Link': links
})
pd.set_option('display.max_colwidth', None)
df_civitatis

,Nombre,Puntuación,Precio (€),Descripción,Link
0,Visita de la Sagrada Familia sin colas,8.6,55,Si hay un lugar que nadie puede perderse en Barcelona es la Sagrada Familia. En esta visita descubriremos la obra maestra de Gaudí saltándonos sus colas.,https://www.civitatis.com//es/barcelona/visita-guiada-sagrada-familia/
1,Free tour por Barcelona,9.5,¡Gratis!,"El free tour por Barcelona es la mejor forma de explorar a pie la capital catalana. Conoceremos las calles, plazas y monumentos imprescindibles de la ciudad.",https://www.civitatis.com//es/barcelona/free-tour-barcelona/
2,Excursión a Montserrat con tren cremallera,8.4,57,"En esta excursión a Montserrat podréis visitar el Santuario de la Patrona de Cataluña, a 1236 metros de altura, y llegar en tren cremallera.",https://www.civitatis.com//es/barcelona/excursion-montserrat/
3,Visita guiada por el Parque Güell,7.7,"25,60",En esta visita guiada por el Parque Güell de Barcelona visitaremos una de las grandes joyas modernistas de Antonio Gaudí en la capital de Cataluña.,https://www.civitatis.com//es/barcelona/visita-guiada-parque-guell/
4,Sagrada Familia con subida a las torres,7.8,70,En esta visita a la Sagrada Familia conoceremos el lugar más famoso de Barcelona y subiremos a una de sus torres. ¡Tendremos una vista inmejorable de la ciudad!,https://www.civitatis.com//es/barcelona/visita-sagrada-familia-torres/
...,...,...,...,...,...
115,Entrada a La Pedrera,9.1,29,"Adentraos en el mundo arquitectónico de Gaudí visitando la Casa Milà, conocida popularmente como La Pedrera. ¡Todo un imprescindible de Barcelona!",https://www.civitatis.com//es/barcelona/entrada-la-pedrera/
116,Tour en tuk tuk por Barcelona,9.5,"53,40",¿Os gustaría visitar las maravillas de Gaudí desde otro punto de vista? En el tour en tuk tuk por Barcelona recorreremos la ciudad de forma cómoda y divertida.,https://www.civitatis.com//es/barcelona/tour-tuk-tuk-barcelona/
117,Paseo en catamarán al atardecer con música en directo,9,"25,50","Contempla el skyline de Barcelona mientras disfrutas de un paseo en catamarán. Además, escucharás música jazz en vivo. ¡Inolvidable!",https://www.civitatis.com//es/barcelona/paseo-catamaran-atardecer/
118,Teleférico de Montjuïc,8.6,16,Con esta entrada al Teleférico de Montjuïc podréis ascender hasta la cima de la montaña con más historia de Barcelona y disfrutar de las vistas de la ciudad.,https://www.civitatis.com//es/barcelona/billete-teleferico-montjuic/
